In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [2]:
from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()


In [3]:
from process_data import get_rest_of_season_player_stats, PRED_COLS
from model_training import get_data_by_windows, get_simple_pipelines, load_player_feature_map

In [59]:
force_retrain = True

player_type = 'goalie'
# X_g, y_g, g_feature_map = get_data_by_windows(player_type, [50, 30, 20, 15, 10, 8, 3, 1])
pipelines = get_simple_pipelines(player_type, (X_g, y_g.loc[X_g.index]), g_feature_map, force_retrain=force_retrain)

player_type = 'skater'
# X, y, full_feature_map = get_data_by_windows(player_type, [30, 15, 10, 5, 3])
pipelines = get_simple_pipelines(player_type, (X, y.loc[X.index]), full_feature_map, force_retrain=force_retrain)


fitting ga
ga -- 0.28972526946089083
ga -- 0.40065374507010043
fitting win
win -- 0.31660576456012657
win -- 0.07152999104436228
fitting so
so -- 0.10124124126963907
so -- 0.012685201349548569
fitting save
save -- 0.34437087193181837
save -- 3.4275143940126074
fitting icetime
icetime -- 0.35785037723553137
icetime -- 0.1276074286424566
fitting g
g -- 0.4141071269918828
g -- 0.06893484371849458
fitting a
a -- 0.4913999081985607
a -- 0.09070259443144378
fitting sog
sog -- 0.7141968560168623
sog -- 0.26608018089710656
fitting fow
fow -- 0.8672928904975596
fow -- 0.33557868474717933
fitting hit
hit -- 0.7380527942445838
hit -- 0.2900396579971427
fitting block
block -- 0.7557504268651606
block -- 0.18557855588215072
fitting pim
pim -- 0.3269239672163742
pim -- 0.16946025636822443
fitting goalsfor
goalsfor -- 0.3503619725348208
goalsfor -- 0.13763992443477568
fitting goalsaga
goalsaga -- 0.3090926791146319
goalsaga -- 0.13380288220262468
fitting ppp
ppp -- 0.5476590049990198
ppp -- 0.0423462

In [ ]:
retrieved data for window 50
retrieved data for window 30
retrieved data for window 20
retrieved data for window 15
retrieved data for window 10
retrieved data for window 8
retrieved data for window 3
retrieved data for window 1
fitting ga
ga -- 0.2897052272708449
ga -- 0.40064544875849517
fitting win
win -- 0.31658687436794275
win -- 0.07153053044298194
fitting so
so -- 0.10125521384063552
so -- 0.012684925207647276
fitting save
save -- 0.3444708019714485
save -- 3.4267838106705746
fitting icetime
icetime -- 0.35784658121593604
icetime -- 0.12760588320019328
retrieved data for window 30
retrieved data for window 15
retrieved data for window 10
retrieved data for window 5
retrieved data for window 3
fitting g
g -- 0.41410366856026026
g -- 0.06893503157785265
fitting a
a -- 0.49140088580460517
a -- 0.09070265491782278
fitting sog
sog -- 0.7141934309633137
sog -- 0.26608251064688077
fitting fow
fow -- 0.867288430127758
fow -- 0.3355780972215602
fitting hit
hit -- 0.738052263806542
hit -- 0.29003904906298844
fitting block
block -- 0.7557499937450254
block -- 0.18557826602146824
fitting pim
pim -- 0.3269243980176004
pim -- 0.16946047164932024
fitting goalsfor
goalsfor -- 0.35036080358785093
goalsfor -- 0.13764021167124563
fitting goalsaga
goalsaga -- 0.309096361647745
goalsaga -- 0.13380339772570363
fitting ppp
ppp -- 0.5476576055823166
ppp -- 0.042346444987147534

In [ ]:
for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(X[skaters_p_feats[col][0]])
preds_df = pd.DataFrame(preds, index=X.index)

gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(X_g[goalies_p_feats[col][0]])
goalie_preds_df = pd.DataFrame(goalie_preds, index=X_g.index)



games_df = pd.DataFrame([dict(week=week, **game) for week, games in current_schedule.items() for game in games])

preds_df = preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(preds_df.index)
goalie_preds_df = goalie_preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(goalie_preds_df.index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds = preds_df[preds_df['week'] > 0].set_index('week', append=True)
weekly_preds = weekly_preds.drop('gameId', axis=1)

weekly_preds.to_csv('data/2324_preds_simple.csv')

In [15]:
import pandas as pd
player_data = pd.read_csv('data/player_data.csv')
player_data.loc[player_data.pos != "['G']"].columns

Index(['playerId', 'name', 'pos', 'team', 'rank', 'week_rank', 'games',
       'week_games', 'g', 'a', 'sog', 'fow', 'hit', 'block', 'pim', 'plusmin',
       'ga', 'win', 'so', 'save', 'g_week', 'a_week', 'sog_week', 'fow_week',
       'hit_week', 'block_week', 'pim_week', 'plusmin_week', 'ga_week',
       'win_week', 'so_week', 'save_week', 'current_lineup', 'selection'],
      dtype='object')

In [ ]:
from yahoo_utils import get_games_by_week
current_schedule = get_games_by_week(427)

In [ ]:
preds = {}
goalie_preds = {}

for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(skater_latest.dropna()[skaters_p_feats[col][0]])
preds_df = pd.DataFrame(preds, index=skater_latest.dropna().index)

gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(goalie_latest.dropna()[goalies_p_feats[col][0]])
goalie_preds_df = pd.DataFrame(goalie_preds, index=goalie_latest.dropna().index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds.to_csv('data/latest_preds.csv')